# Lab - Asociace

## Zadání


_V prostředí Orange vypracujete úlohy 1 a 2 z přiložené prezentace - Analýza nákupního koše (lc-basket-01)._

_V prostředí Jupyter vypracujte úlohy dle prezentace obdobně tak, že na výstupu bude DataFrame obsahující uvedené cílové proměnné pro jednotlivé Frekventované skupiny. Porovnejte výstupy z knihoven mlxtend a apyori._

_Nalezněte nejvíce frekventované skupiny (10) a nejspolehlivější asociační pravidla (10) pro datovou množinu lc-basket-02 v Orange._

_Vypracujte jako zprávu v jupyter notebook nebo jako .md soubor. Nezapomeňte na závěry ke každému bodu a celkový závěr._

## Úloha 1

Z datasetu jsem vyfiltroval pouze sloupce, které se týkají položek nákupního košíku pomocí funkce _Select Columns_.

![Flow](flow.png)

Poté jsem ve funkci _Frequent Itemsets_ nalezl skupiny s minimální velikostí 2 a měli nejvyšší podporu

![Frekventovane_skupiny](freq_g.png)

Dle obrázku je zjevné, že tyto skupiny jsou {cannedveg, frozenmeal} s podporou 17.3%, {frozenmeal, beer} s podporou 17.0% a {cannedveg, beer} s podporou 16.7%

![Rules](rules.png)

Na posledním obrázku, jsem zvolil takové filtrování, kde byla dostatečné velká podpora, v tomto případě 3% zastoupení, abych dostal alespoň 2 pravidla, která zároveň mají spolehlivost alespoň 90%. To se mi povedlo s pravidly _cannedveg, beer, fish -> frozenmeal_ a _cannedmeat, frozenmeal, beer -> cannedveg_ 

Při volbě nižší podpory lze najít i vzorky se 100% spolehlivostí, ale jejich absolutní výskyt je dle mého názoru moc malý.

V druhé úloze jsem frekventované skupiny pormítnul do rozhodovacího stromu, pomocí kterého můžeme vidět, kteří zákazníci s jakými vlastnostmi budou kupovat frekventované skupiny.

![flow2](flow2.png)

V diagramu funkcí, je vidět, že jsem prvně vyfiltroval podstatné sloupce, v tuto chvíli jsem pouze odstranil sloupec cardid, který mi nepřišel relevantní. Dále jsem zvolil řádky, podle toho zda obsahují frekventovanou skupinu a nebo jsem přidal nový slupec, který kategorizoval řádky obsahující frekventovanou skupinu a poté v select column jsem tuto vlastnost označil za Target. Tento postup lze celý vidět v přiloženém souboru _tree.ows_

![tree](tree.png)

__V našem stromu je spoustu interpretovatelných hodnot, ale největší ukazatel pro všechny frekventované skupiny je, že tyto nákupy ve větššině případů provádějí muži s nižším příjmem.__

## Úloha 2

In [86]:
import pandas as pd
from mlxtend.frequent_patterns import apriori

data = pd.read_csv('lc-basket-01 1.tab', sep='\t', skiprows=[1])

In [87]:
data.head()

,cardid,value,pmethod,sex,homeown,income,age,fruitveg,freshmeat,dairy,cannedveg,cannedmeat,frozenmeal,beer,wine,softdrink,fish,confectionery
0,39808,42.7123,CHEQUE,M,NO,27000,46,NaN,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
1,67362,25.3567,CASH,Z,NO,30000,28,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
2,10872,20.6176,CASH,M,NO,13200,36,NaN,NaN,NaN,1.0,NaN,1.0,1.0,NaN,NaN,1.0,NaN
3,26748,23.6883,CARD,Z,NO,12200,26,NaN,NaN,1.0,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN
4,91609,18.8133,CARD,M,YES,11000,24,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [88]:
# Vybereme pouze kategorické proměnné nákupu
item_columns = data.columns[7:]
item_columns

Index(['fruitveg', 'freshmeat', 'dairy', 'cannedveg', 'cannedmeat',
       'frozenmeal', 'beer', 'wine', 'softdrink', 'fish', 'confectionery '],
      dtype='object')

In [89]:
basket_one_hot = data[item_columns].fillna(0).astype(int)
basket_one_hot = basket_one_hot[item_columns].astype(bool)
basket_one_hot.head()

,fruitveg,freshmeat,dairy,cannedveg,cannedmeat,frozenmeal,beer,wine,softdrink,fish,confectionery
0,False,True,True,False,False,False,False,False,False,False,True
1,False,True,False,False,False,False,False,False,False,False,True
2,False,False,False,True,False,True,True,False,False,True,False
3,False,False,True,False,False,False,False,True,False,False,False
4,False,False,False,False,False,False,False,False,False,False,False


In [90]:
frequent_itemsets = apriori(basket_one_hot, min_support=0.1, use_colnames=True)
frequent_itemsets['length'] = frequent_itemsets['itemsets'].apply(lambda x: len(x))

# Tři nejčastější skupiny (min. velikosti 2)
top_3 = frequent_itemsets[frequent_itemsets['length'] >= 2].sort_values(by='support', ascending=False).head(3)

print("Top 3 Frekventované skupiny (lc-basket-01):")
top_3

Top 3 Frekventované skupiny (lc-basket-01):


,support,itemsets,length
12,0.173,"(cannedveg, frozenmeal)",2
14,0.170,"(frozenmeal, beer)",2
13,0.167,"(cannedveg, beer)",2


In [91]:
# Extrahujeme názvy položek z top skupin
group1_items = list(top_3.iloc[0]['itemsets'])
group2_items = list(top_3.iloc[1]['itemsets'])
group3_items = list(top_3.iloc[2]['itemsets'])

group1_items

['cannedveg', 'frozenmeal']

In [92]:

def contains_group(row, items):
    # 1 = nákup, 0/NaN = nenákup
    return all(row[item] == 1 for item in items)

# Vytvoření cílových proměnných (Target Variables)
data['Target_Group_1'] = data.apply(lambda row: contains_group(row, group1_items), axis=1).astype(int)
data['Target_Group_2'] = data.apply(lambda row: contains_group(row, group2_items), axis=1).astype(int)
data['Target_Group_3'] = data.apply(lambda row: contains_group(row, group3_items), axis=1).astype(int)

# Vytvoření finálního DataFrame pro klasifikaci (Features + Targets)
classification_df = data[data.columns[3:7].tolist() + ['Target_Group_1', 'Target_Group_2', 'Target_Group_3']].copy()

print("\nDataFrame s cílovými proměnnými pro klasifikaci:")
print(classification_df.head())
print(classification_df[['Target_Group_1', 'Target_Group_2', 'Target_Group_3']].sum())


DataFrame s cílovými proměnnými pro klasifikaci:
  sex homeown  income  age  Target_Group_1  Target_Group_2  Target_Group_3
0   M      NO   27000   46               0               0               0
1   Z      NO   30000   28               0               0               0
2   M      NO   13200   36               1               1               1
3   Z      NO   12200   26               0               0               0
4   M     YES   11000   24               0               0               0
Target_Group_1    173
Target_Group_2    170
Target_Group_3    167
dtype: int64


Tyto výsledky sedí s těmi z Orange

In [93]:
from apyori import apriori
import pandas as pd

data = pd.read_csv('lc-basket-01 1.tab', sep='\t', skiprows=[1])
data.head()

,cardid,value,pmethod,sex,homeown,income,age,fruitveg,freshmeat,dairy,cannedveg,cannedmeat,frozenmeal,beer,wine,softdrink,fish,confectionery
0,39808,42.7123,CHEQUE,M,NO,27000,46,NaN,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
1,67362,25.3567,CASH,Z,NO,30000,28,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
2,10872,20.6176,CASH,M,NO,13200,36,NaN,NaN,NaN,1.0,NaN,1.0,1.0,NaN,NaN,1.0,NaN
3,26748,23.6883,CARD,Z,NO,12200,26,NaN,NaN,1.0,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN
4,91609,18.8133,CARD,M,YES,11000,24,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [94]:
item_columns = data.columns[7:]
basket_one_hot = data[item_columns].fillna(0).astype(int)
basket_one_hot = basket_one_hot[item_columns].astype(bool)
basket_one_hot.head()

,fruitveg,freshmeat,dairy,cannedveg,cannedmeat,frozenmeal,beer,wine,softdrink,fish,confectionery
0,False,True,True,False,False,False,False,False,False,False,True
1,False,True,False,False,False,False,False,False,False,False,True
2,False,False,False,True,False,True,True,False,False,True,False
3,False,False,True,False,False,False,False,True,False,False,False
4,False,False,False,False,False,False,False,False,False,False,False


In [95]:
# Vytvoříme list listů pro knihovnu apyori
transactions = basket_one_hot.apply(lambda row: basket_one_hot.columns[row == True].tolist(), axis=1)
transactions[999]

['dairy', 'softdrink', 'confectionery ']

In [96]:
frequent_itemsets = apriori(transactions, target = "frequent itemsets",support = 0.1, )
frequent_itemsets = sorted([x for x in frequent_itemsets if len(x.items) > 1], key=lambda x: x.support, reverse=True)[:3]
frequent_itemsets

[RelationRecord(items=frozenset({'cannedveg', 'frozenmeal'}), support=0.173, ordered_statistics=[OrderedStatistic(items_base=frozenset(), items_add=frozenset({'cannedveg', 'frozenmeal'}), confidence=0.173, lift=1.0), OrderedStatistic(items_base=frozenset({'cannedveg'}), items_add=frozenset({'frozenmeal'}), confidence=0.570957095709571, lift=1.890586409634341), OrderedStatistic(items_base=frozenset({'frozenmeal'}), items_add=frozenset({'cannedveg'}), confidence=0.5728476821192052, lift=1.8905864096343408)]),
 RelationRecord(items=frozenset({'frozenmeal', 'beer'}), support=0.17, ordered_statistics=[OrderedStatistic(items_base=frozenset(), items_add=frozenset({'frozenmeal', 'beer'}), confidence=0.17, lift=1.0), OrderedStatistic(items_base=frozenset({'beer'}), items_add=frozenset({'frozenmeal'}), confidence=0.5802047781569967, lift=1.921207874692042), OrderedStatistic(items_base=frozenset({'frozenmeal'}), items_add=frozenset({'beer'}), confidence=0.5629139072847683, lift=1.9212078746920422

Ve výsledcích můžeme opět vidět stejný výsledek, pro všechny knihovny

In [97]:
group1_items = list(frequent_itemsets[0].items)
group2_items = list(frequent_itemsets[1].items)
group3_items = list(frequent_itemsets[2].items)

Část pro DataFrame můžeme kopírovat z předchozí metody

In [98]:
def contains_group(row, items):
    # 1 = nákup, 0/NaN = nenákup
    return all(row[item] == 1 for item in items)

# Vytvoření cílových proměnných (Target Variables)
data['Target_Group_1'] = data.apply(lambda row: contains_group(row, group1_items), axis=1).astype(int)
data['Target_Group_2'] = data.apply(lambda row: contains_group(row, group2_items), axis=1).astype(int)
data['Target_Group_3'] = data.apply(lambda row: contains_group(row, group3_items), axis=1).astype(int)

# Vytvoření finálního DataFrame pro klasifikaci (Features + Targets)
classification_df = data[data.columns[3:7].tolist() + ['Target_Group_1', 'Target_Group_2', 'Target_Group_3']].copy()

print("\nDataFrame s cílovými proměnnými pro klasifikaci:")
print(classification_df.head())
print(classification_df[['Target_Group_1', 'Target_Group_2', 'Target_Group_3']].sum())


DataFrame s cílovými proměnnými pro klasifikaci:
  sex homeown  income  age  Target_Group_1  Target_Group_2  Target_Group_3
0   M      NO   27000   46               0               0               0
1   Z      NO   30000   28               0               0               0
2   M      NO   13200   36               1               1               1
3   Z      NO   12200   26               0               0               0
4   M     YES   11000   24               0               0               0
Target_Group_1    173
Target_Group_2    170
Target_Group_3    167
dtype: int64


Při výběru stejných limitních hodnot podpory, všechny metody měli očekávaně stejné výstupy. 

Lepší pro práci se mi jeví knihovna mlxtend, se kterou se lépe pracuje na bázi pandas DataFrame, ale vesměs je princip jejich práce stejný a dostaneme se ke stejným výsledkům.

U knihovny apyori cenním generátory, jelikož výsledky těchto operací mohou být velmi objemné a zabírat pamět.

## Úloha 3

Abych si ulehčil práci, prvně předělám druhý dataset do stejného formátu jako byl první

In [102]:
import pandas as pd
from mlxtend.frequent_patterns import apriori, association_rules
from mlxtend.preprocessing import TransactionEncoder
import numpy as np

# Load dataset (adjust filename as needed)
df = pd.read_csv("lc-basket-02 1.csv", header=None)

df = df.map(lambda x: str(x).strip().lower() if pd.notna(x) else x)
# Convert rows to lists of items (drop NaNs)
transactions = df.apply(lambda x: x.dropna().tolist(), axis=1).tolist()

# Encode transactions as a one-hot DataFrame
te = TransactionEncoder()
te_ary = te.fit(transactions).transform(transactions)
basket = pd.DataFrame(te_ary, columns=te.columns_)

basket = basket.replace({False: np.nan, True: int(1)})

# Save it for Orange
basket.to_csv("basket_onehot.csv", index=False)

C:\Users\user\AppData\Local\Temp\ipykernel_27140\4128799926.py:18: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  basket = basket.replace({False: np.nan, True: int(1)})


Dle následující tabulky jsou tyto skupiny o velikosti minimálně 2 v top 10

- eggs, mineral water
- eggs, spaghetti
- eggs, french fries
- mineral water, spaghetti
- chocolate, mineral water
- chocolate, spaghetti
- milk, mineral water
- ground beef, mineral water
- ground beef, spaghetti
- frozen vegetables, mineral water

![freq3](freq_basket_2.png)

Toto je 10 nejspolehlivějších pravidel

![pravidla](pravidla.png)

Většina pravidel je přímo viditelně spojena s frekventovanými skupinami, ale některé frekventované skupiny nemusí mít dostatešnou spolehlivost aby se dostali mezi pravidla i když mají větší podporu

## Závěr

V tomto laboratorním cvičení jsme si prakticky ukázali jak se používá metoda Apriori pro asociaci skupin. Můžeme pomocí ni hledat různé vztahy mezi kategorickými hodnotami, které na první pohled nejsou viditelné tak jak je zvykem pro data mining. A celkově jsme mohli vidět jaký je vztah mezi _podporou_ a _spolehlivostí_.